## Prompt Lab Challenge Exercises Notebook

Welcome to the second prompt lab in the bootcamp series, you should have completed lab 1 and the exercises follow on from those. If you completed all the exercises in Lab 1 you should find most of the exercises here straightforward

This notebook is a template with all the exercises and indications of what the output should look like if you do a good job with the prompts.

Before you start you should have a Python environment with the necessary libraries installed as indicated in the intro lab, you will also need a .env file with: 
- your IBM Cloud API key
- the IBM Cloud regional URL (eg, https://us-south.ml.cloud.ibm.com)
- the project ID associated with your WatsonX project (required by the WML Python SDK)

It should take you about 30-45 min to walk through the exercises self paced

Good luck and make sure you compare your answers with the model solutions


In [9]:
import os
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

2. Load credentials for Watsonx.ai (note refer to lab explaining how to do this if necessary)
    - you should have a .env file with your IBM Cloud API key, eg API_KEY=xxx
    - you should have a .env with the IBM Cloud regional url, eg IBM_CLOUD_URL=https://us-south.ml.cloud.ibm.com
    - you should have a .env with the associated WatsonX project ID, eg PROJECT_ID=xxx

In [10]:
#config Watsonx.ai environment

api_key = "Pdw1oZZwH40JCzOBxgNtgQsyXWROMfG2iuyiq6nhljxe"
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
project_id = "838be5fc-4383-452e-937f-4d33516b6232"


if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

Helper function for text generation with the [WML Python SDK](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html) for foundation models.

In [11]:
def send_to_watsonxai(prompts,
                    model_name="google/flan-ul2",
                    decoding_method="greedy",
                    max_new_tokens=100,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0,
                    stop_sequences=[".."]
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES: stop_sequences
    }


    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    #print(prompts)


    for prompt in prompts:
        print(model.generate_text(prompt))

In [12]:
mt_model = 'bigscience/mt0-xxl'
flanul_model = 'google/flan-ul2'
granite_13b_instruct_v2 = 'ibm/granite-13b-instruct-v2'
llama2 = "meta-llama/llama-2-70b-chat"
t5 = "google/flan-t5-xxl"

---

In [13]:
# Product Review for Questions  1 - 5
lamp_review = """Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!"""

#### Q1) write a prompt to return the sentiment of the review
Target sentiment = positive

In [14]:
#Q1 Code - enter prompt and function parameters in this cell
prompt = f"identify the sentiment in the following product review: {lamp_review}"
send_to_watsonxai(prompts=[prompt],
                  min_new_tokens= 1,
                  max_new_tokens= 2)

positive


#### Q2) extract the emotions the reviewer expressed, return answer as a comma separated list
Target emotions = satisfied, happy, cared for, great company, product!

In [15]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],min_new_tokens=10,max_new_tokens=30)

satisfied, happy, cared for, happy, satisfied


#### Q3) Is the reviewer expressing anger, answer “yes” or “no” – test with your own example including anger to ensure it works in both cases.
Target answer = no

In [16]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''

"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=2,min_new_tokens=1)

no


#### Q4) Extract the item purchased and the company name, return as JSON format
Target answer = Item[lamp], Brand[Lumina]

In [17]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=30,min_new_tokens=10)

Item[lamp], Brand[Lumina]


#### Q5) Can you combine 3-6 in a single prompt and return JSON with: Sentiment (negative or positive), Anger (yes/no), Product, Company
Target answer = Sentiment[positive], Anger[false], Item[lamp], Brand[Lumina]

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50,min_new_tokens=20)


Sentiment[positive], Anger[false], Item[lamp], Brand[Lumina]


#### Q6) summarize the following product review
Example summary = My daughter loves it!  It's soft and  super cute, and its face has a friendly look. It's  a bit small for what I paid though.

In [19]:
# Product Review for Questions  6 - 8
review = """Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her."""

In [20]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50,min_new_tokens=10)

My daughter loves it! but it's a bit small for what I paid


#### Q7) Summarize the same product review from the perspective of the shipping department
Example summary = It arrived a day earlier than expected, so I got to play with it myself before I gave it  to her. 

In [21]:
#concise wrt feedback shipping
prompt = f""" 

read the review below, highlighted between triple backticks, and tell me about the delivery of the product

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=50, min_new_tokens=10)

It arrived a day earlier than expected,  so I got to play with it myself before I gave it to her.


#### Q8) Summarize the review from the perspective of pricing and value
Example summary = It's a bit small for what I paid though. I think there might be other options that are bigger for the same price

In [22]:
#feedback pricing works - concise
prompt = f"""
You should read the review below, delimited by triple 
backticks, summarize any aspects \
that are relevant to the price and perceived value.  

Review: ```{review}```
"""

send_to_watsonxai(prompts=[prompt],min_new_tokens=10,max_new_tokens=50)


It's a bit small for what I paid though. I think there  might be other options that are bigger for the  same price


---

#### Q9)	PII removal. Given the following email, write a prompt to remove the PII (eg names, emails etc) (Hint: you may need to use 1-2 shot technique)

In [24]:
email = """
Hi John,\

I'm writing to you because I noticed you recently purchased a new car. I'm a salesperson\
at a local dealership (Cheap Dealz), and I wanted to let you know that we have a great deal on a new\
car. If you're interested, please let me know.\

Thanks,\

Jimmy Smith\

Phone: 410-805-2345\
Email: jimmysmith@cheapdealz.com\
"""

Hint - use prompt template or manually construct the prompt with f strings (look up in documentation if unsure)

In [25]:
prompt = f"""Given the following text, remove the PII (eg names, emails etc):

Input: Hello, my name is Ahmed, and this is my email: ggg@xyx.com

Output: Hello, my name is name, and this is my email: email

Input: Hello Lucy, I wish you are fine. You know me, I'm Sandy, your friend form school. Let's connect wjen you have some time on my email: SANDY@uuu.com. Thanks Sandy Kevin.

Output: Hello name, I wish you are fine. You know me, I'm name, your friend form school. Let's connect wjen you have some time on my email: email. Thanks, name name.

Input: '''{email}'''

Output:
"""
response = send_to_watsonxai(prompts=[prompt], model_name="ibm-mistralai/mixtral-8x7b-instruct-v01-q")


Hi name,
I'm writing to you because I noticed you recently purchased a new car. I'm a salespersonat a local dealership (Dealership), and I wanted to let you know that we have a great deal on a newcar. If you're interested, please let me know.
Thanks,
name name
Phone: Phone
Email: Email


---

#### Q10) Basic inference: A patients a1c level determines their diabetes status, the rules are as follows:

 - less than 5.7 no diabetes
 - between 5.7 and 6.5 pre-diabetes
 - greater than 6.5 diabetic.

Write a prompt to return just the diabetes status from the following 3 test cases:

1)	The patients a1c is 5.5 which is good considering his other risk factors.
2)	From the last lab report I noted the A1c is 6.4 so we need to put her on Ozempic.
3)	She mentioned her A1c is 8 according to her blood work about 3 years ago.

Bonus 1: How could you improve the inference given the other information in the sentences?

Bonus 2: how would you approach extracting the diabetes status based on patient notes without A1C values and what would you need to watch out for? (hint: maybe they are talking about family history of disease or other complications)


In [26]:
record1 = "The patients a1c is 5.5 which is good considering his other risk factors."
record2 = "From the last lab report I noted the A1c is 6.4 so we need to put her on Ozempic."
record3 = "She mentioned her A1c is 8 according to her blood work about 3 years ago."

In [32]:
#Q1 ENTER YOUR MODEL PARAMS HERE - MAKE SURE IT WORKS WITH ALL 3 EXAMPLES ABOVE
prompt = f"""Diabetes status is indicated by a patients a1c level. the following a1c ranges determine the diabetes status.
First, read the a1c level from the text delimited by triple back-ticks.

Next, if the a1c level is less than 5.7 then output diabetes status: no-diabetes
if a1c level is greater than 6.5: then output diabetes status: diabetes
if the a1c is between 5.8 and 6.4, then output diabetes status: pre-diabetes

```She mentioned her A1c is 3 according to her blood work about 3 years ago.```

what is the diabetes status:"""

send_to_watsonxai(prompts=[prompt],max_new_tokens=10,min_new_tokens=1,model_name="ibm-mistralai/mixtral-8x7b-instruct-v01-q")

 no-diabetes
